In [1]:
using Pkg
Pkg.activate(".")
# Pkg.instantiate()
using Revise
using EasyHybrid
using Lux
using Optimisers
using WGLMakie
using Random
using LuxCore
using CSV, DataFrames
using EasyHybrid.MLUtils
using Statistics
using Plots
using JLD2


  Activating project at `/mnt/tupi/HybridModeling/EasyDensity.jl`


In [2]:
# 03 - flexiable BD, both oBD and mBD will be learnt by NN
testid = "03_hybridNN";
version = "v20251125"
results_dir = joinpath(@__DIR__, "eval");
target_names = [:BD, :SOCconc, :CF, :SOCdensity];

# input
df = CSV.read(joinpath(@__DIR__, "data/lucas_preprocessed_$version.csv"), DataFrame; normalizenames=true)
println(size(df))

# scales
scalers = Dict(
    :SOCconc   => 0.151, # g/kg, log(x+1)*0.151
    :CF        => 0.263, # percent, log(x+1)*0.263
    :BD        => 0.529, # g/cm3, x*0.529
    :SOCdensity => 0.167, # kg/m3, log(x)*0.167
);

# mechanistic model
function SOCD_model(; SOCconc, CF, oBD, mBD)
    ϵ = 1e-7

    # invert transforms
    soct = (exp.(SOCconc ./ scalers[:SOCconc]) .- 1) ./ 1000
    soct = clamp.(soct, ϵ, Inf)

    cft = (exp.(CF ./ scalers[:CF]) .- 1) ./ 100
    cft = clamp.(cft, 0, 0.99)

    # compute BD safely
    som = 1.724f0 .* soct
    denom = som .* mBD .+ (1f0 .- som) .* oBD
    denom = clamp.(denom, ϵ, Inf)

    BD = (oBD .* mBD) ./ denom
    BD = clamp.(BD, ϵ, Inf)

    # SOCdensity
    SOCdensity = soct .* 1000 .* BD .* (1 .- cft)
    SOCdensity = clamp.(SOCdensity, ϵ, Inf)

    # scale
    SOCdensity = log.(SOCdensity) .* scalers[:SOCdensity]
    BD = BD .* scalers[:BD]

    return (; BD, SOCconc, CF, SOCdensity, oBD, mBD)
end


# param bounds
parameters = (
    SOCconc = (0.01f0, 0.0f0, 1.0f0),   # fraction
    CF      = (0.15f0, 0.0f0, 1.0f0),   # fraction,
    oBD     = (0.20f0, 0.05f0, 0.40f0),  # also NN learnt, g/cm3
    mBD     = (1.20f0, 0.75f0, 2.0f0),  # NN leanrt
)

# define param for hybrid model
neural_param_names = [:SOCconc, :CF, :mBD, :oBD]
# global_param_names = [:oBD]
forcing = Symbol[]
targets = [:BD, :SOCconc, :SOCdensity, :CF]       # SOCconc is both a param and a target

# predictor
predictors = Symbol.(names(df))[18:end-6]; # CHECK EVERY TIME 
nf = length(predictors)


(56117, 385)


362

In [3]:
# hyperparameters
# search space
hidden_configs = [ 
    (512, 256, 128, 64, 32, 16),
    (512, 256, 128, 64, 32), 
    (256, 128, 64, 32, 16),
    (256, 128, 64, 32),
    (256, 128, 64),
    (128, 64, 32, 16),
    (128, 64, 32),
    (64, 32, 16)
];
batch_sizes = [128, 256, 512];
lrs = [1e-3, 5e-4, 1e-4];
activations = [relu, swish, gelu];

configs = [(h=h, bs=bs, lr=lr, act=act)
           for h in hidden_configs
           for bs in batch_sizes
           for lr in lrs
           for act in activations]

println(length(configs))


216


In [4]:
# cross-validation
k = 5;
folds = make_folds(df, k = k, shuffle = true);
rlt_list_param = Vector{DataFrame}(undef, k)
rlt_list_pred = Vector{DataFrame}(undef, k)  

@info "Threads available: $(Threads.nthreads())"

[ Info: Threads available: 96


In [ ]:

@time for test_fold in 1:k
    @info "Training outer fold $test_fold of $k"

    train_folds = setdiff(1:k, test_fold)
    train_idx = findall(in(train_folds), folds)
    train_df = df[train_idx, :]
    test_idx  = findall(==(test_fold), folds)
    test_df = df[test_idx, :]

    # track best config for this outer fold
    lk = ReentrantLock()
    best_val_loss = Inf
    best_config = nothing
    best_result = nothing
    best_model_path = nothing
    best_model = nothing

    Threads.@threads for i in 1:(length(configs))
        try
            cfg = configs[i]
        
            h  = cfg.h
            bs = cfg.bs
            lr = cfg.lr
            act = cfg.act
            println("Testing h=$h, bs=$bs, lr=$lr, activation=$act")
    
            hm_local = constructHybridModel(
                predictors,
                forcing,
                targets,
                SOCD_model,
                parameters,
                neural_param_names,
                [];
                hidden_layers = collect(h),
                activation = act,
                scale_nn_outputs = true,
                input_batchnorm = false,
                start_from_default = true
            )
    
            rlt = train(
                hm_local, train_df, ();
                nepochs = 200,
                batchsize = bs,
                opt = AdamW(lr),
                training_loss = :mse,
                loss_types = [:mse, :r2],
                shuffleobs = true,
                file_name = "$(testid)_config$(i)_fold$(test_fold).jld2",
                random_seed = 42,
                patience = 15,
                yscale = identity,
                monitor_names = [:oBD, :mBD],
                agg = mean,
                return_model = :best,
                show_progress = false,
                plotting = false,
                hybrid_name = "$(testid)_config$(i)_fold$(test_fold)" 
            )
    
            lock(lk)
            if rlt.best_loss < best_val_loss
                best_val_loss = rlt.best_loss
                best_config = cfg
                best_result = rlt
                best_model_path = "best_model_$(testid)_config$(i)_fold$(test_fold).jld2"
                best_model = deepcopy(hm_local)
            end
            unlock(lk)
        catch err
            @error "Thread $i crashed" exception = err
            @error sprint(showerror, err)
        end

    end

    # register best hyper paramets
    agg_name = Symbol("mean")
    r2s  = map(vh -> getproperty(vh, agg_name), best_result.val_history.r2)
    mses = map(vh -> getproperty(vh, agg_name), best_result.val_history.mse)
    best_epoch = max(best_result.best_epoch, 1)

    local_results_param = DataFrame(
        h = string(best_config.h),
        bs = best_config.bs,
        lr = best_config.lr,
        act = string(best_config.act),
        r2 = r2s[best_epoch],
        mse = mses[best_epoch],
        best_epoch = best_epoch,
        test_fold = test_fold,
        path = best_model_path,
    )
    rlt_list_param[test_fold] = local_results_param

    ps, st = best_result.ps, best_result.st
    (x_test,  y_test)  = prepare_data(best_model, test_df)
    ŷ_test, st_test = best_model(x_test, ps, LuxCore.testmode(st))
    # println(propertynames(ŷ_test))
    # println(propertynames(ŷ_test.parameters))

    for var in [:BD, :SOCconc, :CF, :SOCdensity, :oBD, :mBD]
        if hasproperty(ŷ_test, var)
            val = getproperty(ŷ_test, var)

            if val isa AbstractVector && length(val) == nrow(test_df)
                test_df[!, Symbol("pred_", var)] = val # per row

            elseif (val isa Number) || (val isa AbstractVector && length(val) == 1)
                test_df[!, Symbol("pred_", var)] = fill(Float32(val isa AbstractVector ? first(val) : val), nrow(test_df))
            end


        end
    end
    
    rlt_list_pred[test_fold] = test_df

end

rlt_param = vcat(rlt_list_param...)
rlt_pred = vcat(rlt_list_pred...)

CSV.write(joinpath(results_dir, "$(testid)_cv.pred_$version.csv"), rlt_pred)
CSV.write(joinpath(results_dir, "$(testid)_hyperparams_$version.csv"), rlt_param)


[ Info: Training outer fold 1 of 5


Testing h=(256, 128, 64), bs=512, lr=0.0005, activation=swish
Testing h=(64, 32, 16), bs=128, lr=0.0005, activation=relu
Testing h=(512, 256, 128, 64, 32), bs=256, lr=0.0005, activation=relu
Testing h=(512, 256, 128, 64, 32), bs=512, lr=0.001, activation=relu
Testing h=(128, 64, 32), bs=128, lr=0.0005, activation=swish
Testing h=(256, 128, 64, 32), bs=512, lr=0.0001, activation=swish
Testing h=(256, 128, 64, 32, 16), bs=512, lr=0.001, activation=relu
Testing h=(256, 128, 64), bs=128, lr=0.001, activation=gelu_tanh
Testing h=(128, 64, 32, 16), bs=256, lr=0.001, activation=gelu_tanh
Testing h=(512, 256, 128, 64, 32), bs=512, lr=0.0001, activation=relu
Testing h=(64, 32, 16), bs=256, lr=0.0005, activation=swish
Testing h=(128, 64, 32, 16), bs=256, lr=0.0001, activation=gelu_tanh
Testing h=(512, 256, 128, 64, 32), bs=128, lr=0.001, activation=relu
Testing h=(512, 256, 128, 64, 32), bs=512, lr=0.0005, activation=relu
Testing h=(64, 32, 16), bs=512, lr=0.001, activation=swish
Testing h=(256,

[ Info: Plotting disabled.
[ Info: Plotting disabled.
[ Info: Plotting disabled.
[ Info: Plotting disabled.
[ Info: Plotting disabled.
[ Info: Plotting disabled.
[ Info: Plotting disabled.
[ Info: Plotting disabled.
[ Info: Plotting disabled.
[ Info: Plotting disabled.
[ Info: Plotting disabled.
[ Info: Plotting disabled.
[ Info: Plotting disabled.
[ Info: Plotting disabled.
[ Info: Plotting disabled.
[ Info: Plotting disabled.
[ Info: Plotting disabled.
[ Info: Plotting disabled.
[ Info: Plotting disabled.
[ Info: Plotting disabled.
[ Info: Plotting disabled.
[ Info: Plotting disabled.
[ Info: Plotting disabled.
[ Info: Plotting disabled.
[ Info: Plotting disabled.
[ Info: Plotting disabled.
[ Info: Plotting disabled.
[ Info: Plotting disabled.
[ Info: Plotting disabled.
[ Info: Plotting disabled.
[ Info: Plotting disabled.
[ Info: Plotting disabled.
[ Info: Plotting disabled.
[ Info: Plotting disabled.
[ Info: Plotting disabled.
[ Info: Plotting disabled.
[ Info: Plotting disabled.
[

Testing h=(256, 128, 64, 32, 16), bs=512, lr=0.0005, activation=swish


[ Info: Check the saved output (.png, .mp4, .jld2) from training at: /mnt/tupi/HybridModeling/EasyDensity.jl/output_tmp
[ Info: Check the saved output (.png, .mp4, .jld2) from training at: /mnt/tupi/HybridModeling/EasyDensity.jl/output_tmp
[ Info: Check the saved output (.png, .mp4, .jld2) from training at: /mnt/tupi/HybridModeling/EasyDensity.jl/output_tmp
[ Info: Check the saved output (.png, .mp4, .jld2) from training at: /mnt/tupi/HybridModeling/EasyDensity.jl/output_tmp
[ Info: Check the saved output (.png, .mp4, .jld2) from training at: /mnt/tupi/HybridModeling/EasyDensity.jl/output_tmp
[ Info: Check the saved output (.png, .mp4, .jld2) from training at: /mnt/tupi/HybridModeling/EasyDensity.jl/output_tmp
[ Info: Check the saved output (.png, .mp4, .jld2) from training at: /mnt/tupi/HybridModeling/EasyDensity.jl/output_tmp
[ Info: Check the saved output (.png, .mp4, .jld2) from training at: /mnt/tupi/HybridModeling/EasyDensity.jl/output_tmp
[ Info: Check the saved output (.png, .m

...

Testing h=(128, 64, 32), bs=128, lr=0.001, activation=swish


[ Info: Plotting disabled.
┌ Warning: Early stopping at epoch 20 with best validation loss wrt mse: 0.014082277828947198
└ @ EasyHybrid /opt/julia/packages/EasyHybrid/n8FOE/src/train.jl:273
[ Info: Returning best model from epoch 5 of 200 epochs with best validation loss wrt mse: 0.014082277828947198
[ Info: Check the saved output (.png, .mp4, .jld2) from training at: /mnt/tupi/HybridModeling/EasyDensity.jl/output_tmp
┌ Warning: Early stopping at epoch 19 with best validation loss wrt mse: 0.013944239030887953
└ @ EasyHybrid /opt/julia/packages/EasyHybrid/n8FOE/src/train.jl:273
[ Info: Returning best model from epoch 4 of 200 epochs with best validation loss wrt mse: 0.013944239030887953
┌ Warning: Early stopping at epoch 32 with best validation loss wrt mse: 0.020077122858022163
└ @ EasyHybrid /opt/julia/packages/EasyHybrid/n8FOE/src/train.jl:273
[ Info: Returning best model from epoch 17 of 200 epochs with best validation loss wrt mse: 0.020077122858022163


Testing h=(128, 64, 32), bs=128, lr=0.0001, activation=swish


[ Info: Plotting disabled.
Excessive output truncated after 524374 bytes.

"/mnt/tupi/HybridModeling/EasyDensity.jl/eval/03_hybridNN_hyperparams_v20251125.csv"

In [ ]:
bad_rows = rlt_pred[rlt_pred.SOCdensity .< 0, :]


In [ ]:
println(sizeof(rlt_pred))